In [39]:
import mne
import numpy as np
from mne.preprocessing import (ICA)
from autoreject import AutoReject
from scipy.signal import find_peaks
import matplotlib
matplotlib.use("TkAgg")

eeg_path = "/Users/ansle/Documents/GitHub/undergrad_files/"  # You will need to change this location
file_name = "PhotoDiode_2024-03-22_10-24-46"
file_eeg = eeg_path + file_name + ".eeg"
file_vhdr = eeg_path + file_name + ".vhdr"
file_vmrk = eeg_path + file_name + ".vmrk"

# Reload the EEG data now that all necessary files are available
raw = mne.io.read_raw_brainvision(file_vhdr)
raw.crop(tmin=22, tmax=190)
#raw.plot()

Extracting parameters from /Users/ansle/Documents/GitHub/undergrad_files/PhotoDiode_2024-03-22_10-24-46.vhdr...
Setting channel info structure...


/var/folders/dv/s4k5_3r51ml3q36bgbcdsgbr0000gn/T/ipykernel_27181/3688953752.py:16: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_brainvision(file_vhdr)


<RawBrainVision | PhotoDiode_2024-03-22_10-24-46.eeg, 1 x 84001 (168.0 s), ~6 kB, data not loaded>

In [40]:
# Find events again
events, event_id = mne.events_from_annotations(raw)

# Filter to keep only Stimulus/s2 events
stimulus_s1_events = events[events[:, 2] == event_id['Stimulus/s1']]
stimulus_s1_events, event_id

# Define the epoch parameters
tmin, tmax = -0.250, 0.250  # start and end time around each event in seconds

# Create epochs around the Stimulus/s2 events
epochs = mne.Epochs(raw, events=stimulus_s1_events, event_id=event_id['Stimulus/s1'],
                    tmin=tmin, tmax=tmax, preload=True)

# Pick the BIP3 channel
epochs = epochs.pick_channels(['BIP3'])

# Calculate the peak latency for each epoch
peak_latencies = []
for epoch in epochs.get_data():
    # Get the time index and amplitude of the peak
    peak_sample = epoch.argmax()
    peak_time = epochs.times[peak_sample]
    peak_latencies.append(peak_time)


Used Annotations descriptions: ['Stimulus/s1', 'Stimulus/s2', 'Stimulus/s3', 'Stimulus/s5']
Not setting metadata
100 matching events found
Setting baseline interval to [-0.25, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 100 events and 251 original time points ...
0 bad epochs dropped


In [ ]:
# PLOT LATENCIES
import numpy as np
import matplotlib.pyplot as plt

# Calculate the average latency
average_latency = np.mean(peak_latencies)

# Visualize the latency distribution
plt.figure(figsize=(10, 6))
plt.hist(peak_latencies, bins=20, color='skyblue', edgecolor='black')
plt.axvline(average_latency, color='red', linestyle='dashed', linewidth=2)
plt.title('Distribution of Maximum Photodiode Peaks (-0.250, 0.250)')
plt.xlabel('Latency (seconds)')
plt.ylabel('Frequency')
plt.legend(['Average Latency', 'Latencies'])
plt.grid(True)

plt.show(), average_latency


In [ ]:
print(average_latency)
total_events = len(peak_latencies)
total_events
#Image changes 0.09228 seconds after the event marker, 100 events

In [28]:
# Update the epoch parameters for the new time window
tmin_updated, tmax_updated = -0.250, 0.250
amplitude_threshold = 125e-6

# Recreate epochs with the updated time window
epochs_updated = mne.Epochs(raw, events=stimulus_s1_events, event_id=event_id['Stimulus/s1'],
                            tmin=tmin_updated, tmax=tmax_updated, preload=True)

# Pick the BIP3 channel again
epochs_updated = epochs_updated.pick_channels(['BIP3'])

# Calculate the index for the updated stimulus onset
stimulus_onset_index_updated = np.where(epochs_updated.times == 0)[0][0]

# Find the first timepoint exceeding 200 µV within the new time window for each epoch
exceeding_threshold_latencies_updated = []
for epoch in epochs_updated.get_data():
    # Find the index where the amplitude first exceeds 200 µV
    exceeding_index_updated = np.where(epoch[0, :] > amplitude_threshold)[0]
    if exceeding_index_updated.size > 0:  # Check if there is at least one exceeding point
        # Get the time for the first exceeding point
        first_exceeding_time_updated = epochs_updated.times[exceeding_index_updated[0]]
        exceeding_threshold_latencies_updated.append(first_exceeding_time_updated)
    else:
        # If no points exceed the threshold, append None
        exceeding_threshold_latencies_updated.append(None)

# Filter out None values for calculating average and visualization
valid_latencies_updated = [latency for latency in exceeding_threshold_latencies_updated if latency is not None]

# Calculate the average latency of valid latencies within the new time window
average_latency_threshold_updated = np.mean(valid_latencies_updated) if valid_latencies_updated else 0

len(valid_latencies_updated), average_latency_threshold_updated


Not setting metadata
100 matching events found
Setting baseline interval to [-0.25, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 100 events and 251 original time points ...
0 bad epochs dropped


(87, 0.08809195402298849)

In [9]:
# Visualize the distribution of valid latencies within the updated time window
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(valid_latencies_updated, bins=20, color='orange', edgecolor='black')
plt.axvline(average_latency_threshold_updated, color='blue', linestyle='dashed', linewidth=2)
plt.title('Distribution of Photodiode Peaks >200 uV (-0.250, 0.250)')
plt.xlabel('Latency (seconds)')
plt.ylabel('Frequency')
plt.legend(['Average Latency', 'Latencies'])
plt.grid(True)

plt.show()


In [29]:
import mne
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'raw' and 'event_id' are already defined and loaded

# Update the epoch parameters for the new time window and amplitude threshold
tmin_updated, tmax_updated = 0, 0.250
amplitude_threshold = 125e-6  # 125 µV in Volts

# Assuming you have the 'Stimulus/s1' events extracted somewhere above as 'stimulus_s1_events'
# If not, you'll need to extract them similar to how 'Stimulus/s2' events were extracted:
# events, event_id = mne.events_from_annotations(raw)
# stimulus_s1_events = events[events[:, 2] == event_id['Stimulus/s1']]

# Recreate epochs for Stimulus/s1 within the specified window
epochs_updated = mne.Epochs(raw, events=stimulus_s1_events, event_id=event_id['Stimulus/s1'],
                            tmin=tmin_updated, tmax=tmax_updated, preload=True, baseline=None)

# Pick the BIP3 channel
epochs_updated = epochs_updated.pick_channels(['BIP3'])

# Find the first timepoint exceeding 125 µV after the stimulus in each epoch
exceeding_threshold_latencies_updated = []
for epoch in epochs_updated.get_data():
    # Find the index where the amplitude first exceeds 125 µV
    exceeding_index_updated = np.where(epoch[0, :] > amplitude_threshold)[0]
    if exceeding_index_updated.size > 0:  # Check if there's at least one exceeding point
        first_exceeding_time_updated = epochs_updated.times[exceeding_index_updated[0]]
        exceeding_threshold_latencies_updated.append(first_exceeding_time_updated)
    else:
        exceeding_threshold_latencies_updated.append(None)

# Filter out None values for calculating average and visualization
valid_latencies_updated = [latency for latency in exceeding_threshold_latencies_updated if latency is not None]

# Calculate the average latency of valid latencies
average_latency_threshold_updated = np.mean(valid_latencies_updated) if valid_latencies_updated else 0

print(f"Number of valid latencies: {len(valid_latencies_updated)}, Average latency: {average_latency_threshold_updated}")

# Visualize the distribution of valid latencies
plt.figure(figsize=(10, 6))
plt.hist(valid_latencies_updated, bins=20, color='orange', edgecolor='black')
plt.axvline(average_latency_threshold_updated, color='blue', linestyle='dashed', linewidth=2)
plt.title('Distribution of Photodiode Peaks >125 uV (0, 0.250)')
plt.xlabel('Latency (seconds)')
plt.ylabel('Frequency')
plt.legend(['Average Latency', 'Latencies'])
plt.grid(True)

plt.show()


Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
Loading data for 100 events and 126 original time points ...
0 bad epochs dropped
Number of valid latencies: 56, Average latency: 0.09949999999999999


In [37]:
epochs_time_windowed = mne.Epochs(raw, events=stimulus_s1_events, event_id=event_id['Stimulus/s1'],
                                  tmin=-0.250, tmax=0.250, preload=True, baseline=(None, None))

epochs_time_windowed = epochs_time_windowed.pick_channels(['BIP3'])

epochs_time_windowed.plot_image(picks=['BIP3'], sigma=1.0, cmap='viridis', 
                          vmin=-125, vmax=125)


Not setting metadata
100 matching events found
Setting baseline interval to [-0.25, 0.25] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 100 events and 251 original time points ...
0 bad epochs dropped
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped


/var/folders/dv/s4k5_3r51ml3q36bgbcdsgbr0000gn/T/ipykernel_27181/1168277652.py:6: RuntimeWarning: Cannot find channel coordinates in the supplied Evokeds. Not showing channel locations.
  epochs_time_windowed.plot_image(picks=['BIP3'], sigma=1.0, cmap='viridis',


[<Figure size 480x360 with 3 Axes>]

In [38]:
epochs_time_windowed = mne.Epochs(raw, events=stimulus_s1_events, event_id=event_id['Stimulus/s3'],
                                  tmin=-0.250, tmax=0.250, preload=True, baseline=(None, None))

epochs_time_windowed = epochs_time_windowed.pick_channels(['BIP3'])

epochs_time_windowed.plot_image(picks=['BIP3'], sigma=1.0, cmap='viridis', 
                          vmin=-125, vmax=125)


ValueError: No matching events found for 10003 (event id 10003)